# TP2
### Équipe 10
### Fabrice Charbonneau (1798064)
### Sanou Armel Kenzanga Landry (1976804)


1. créer une matrice termes-documents; pour diminuer la taille du vocabulaire, utiliser un stemmer et éliminer les chiffres et la ponctuation;
2. transformer les fréquences brutes par les valeurs TF-IDF;
3. effectuer une réduction de dimensions avec SVD; utilisez 50 dimensions latentes.



D'abord, lire les fichiers et les ajouter à un dataframe.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import PlaintextCorpusReader
from os import path

In [ ]:
if not path.exists("raw_data.csv"):
    i = 0
    df = pd.DataFrame(columns=['uni', 'course', 'text'])
    class_data = 'Descriptions' 
    corpus = PlaintextCorpusReader(class_data, '.*txt', encoding='latin-1')
    for file in corpus.fileids():
        uni_str = file.split("/")[0]
        new_corpus = PlaintextCorpusReader(class_data, file, encoding='latin-1')
        df.loc[i] = [uni_str, file.split("/")[1].split(".")[0], new_corpus.raw()]
        i += 1
        print(i,  end='\r')
    df.to_csv("raw_data.csv", index=False)

In [ ]:
df = pd.read_csv("raw_data.csv")

Détection du langage des textes avec https://pypi.org/project/langdetect/

Nous n'allons conserver que les cours ayant comme langage détecté le français.

In [ ]:
from langdetect import detect

if not path.exists("fr_only.csv"):
    df["lang"] = df.apply( lambda row: detect(row["text"]), axis=1)
    df = df[df["lang"] == "fr"]
    df.to_csv("fr_only.csv", index=False)
df = pd.read_csv("fr_only.csv")

In [ ]:
df = df.drop(columns="lang")

In [ ]:
df

Utilisation d'un Tokenizer pour séparer les strings en mots.

TweetTokenizer de nltk est à la base fait pour les tweets, donc traite le texte pouvant contenir des hashtags et autres particularité des tweets. Dans notre cas, ça ne change pas grand chose, il fera facilement le travail même le texte n'est pas des tweets.

In [ ]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
tokens = df["text"].apply(lambda x: tknzr.tokenize(x)).values

Stemmer afin d'éliminer les tokens inutles

In [ ]:
from nltk.stem.snowball import SnowballStemmer
import string
from nltk.corpus import stopwords

In [ ]:
st = SnowballStemmer("french", ignore_stopwords=True)
[st.stem(word) for word in ["Yeux", "Sociologie", ".", "souris"]]

In [ ]:
class Preprocessor:
    def __init__(self, stem=True, punctuation=True, digit=True):
        self.st = SnowballStemmer("french", ignore_stopwords=True)
        self.punctuations = set(string.punctuation)
        self.stem = stem
        self.punctuation = punctuation
        self.digit = digit
    
    def raw_preprocess(self, token_list):
        new_token_list = []
        for token in token_list:
            is_token_valid = True
            token = token.lower()
            
            # stemming
            if self.stem:
                token = self.st.stem(token)

            # punctuation filtering
            if self.punctuation and token in self.punctuations:
                is_token_valid = False

            # digit filtering
            elif self.digit and token.isdigit():
                is_token_valid = False

            # add token to new sub list
            if is_token_valid:
                new_token_list.append(token)
        return new_token_list
        
    def preprocess_tokens(self, tokens):
        new_tokens = []
        for token_list in tokens:
            new_tokens.append(self.raw_preprocess(token_list))
        return new_tokens
    
def preprocess(tokens):
    preprocessor = Preprocessor()
    return preprocessor.preprocess_tokens(test)

In [ ]:
test = [
    ['Sociologie', 'de', 'l', 'DescriptionCours', ':', 'Trois', 'volets', 'principaux', 'dans', 'ce', 'cours'],
    ['Management', 'DescriptionCours', ':', 'Ce', 'cours', 'est', 'une', 'introduction', 'au', 'management', 'et']
]
preprocessor = Preprocessor()
preproc_test = preprocessor.preprocess_tokens(test)
preproc_test

TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def identity_tokenizer(text):
    return text

#vectorizer = TfidfVectorizer(preprocessor = preprocessor.preprocess_tokens, tokenizer=identity_tokenizer)
vectorizer = TfidfVectorizer(preprocessor=preprocessor.raw_preprocess ,tokenizer=identity_tokenizer)
IDF_data = vectorizer.fit_transform(tokens)

SVD

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=50, algorithm='randomized', n_iter=100)
svd.fit(IDF_data)

In [ ]:
sum(svd.explained_variance_)

In [ ]:
SVD_data = svd.transform(IDF_data)

## 1. Quels sont les 10 cours les plus similaires à LOG2420 dans l'espace réduit à 50 dimensions? (8 pts.) Prenez le cosinus comme mesure de similarité. 

In [ ]:
LOG2420_idx = df.index[df["course"] == 'LOG2420'][0]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_matrix = cosine_similarity(SVD_data)

In [ ]:
log_2420_cos = cos_matrix[LOG2420_idx]
log_2420_sorted_cos = np.sort(log_2420_cos)
recommendations_cos = log_2420_sorted_cos[::-1][1:11]

In [ ]:
recommendations_indx = [np.where(log_2420_cos == reco)[0][0] for reco in recommendations_cos]

In [ ]:
df.loc[list(recommendations_indx)]

## 2. Effectuez une classification de cours par une approche supervisée (4 pts.) Pour établir les libellés de classes, prenez les acronymes de cours commençant par PSY (psychologie) et PHY (physique). Calculez les centroides de chaque classe (PSY et PHY) des données d'entraînement et calculez l'aire sous la courbe ROC pour la prédiction de données de test (AUC, Area Under receiver operator Curve). 

In [ ]:
mask = ((df["course"].astype(str).str[:3] == "PSY") | (df["course"].astype(str).str[:3] == "PHY"))
train_df = df[mask]

train_df["PSY"] = df["course"].astype(str).str[:3] == "PSY"
train_df

## 3. À partir de la validation croisée de la tâche de classification ci-dessus, déterminez le nombre de dimensions latentes optimal de SVD selon une approche dite wrapper (4 pts.).

## 4.Effectuez une agglomération par k-means (k=2) et vérifiez si les classes PHY et PSY sont bien séparées par cette méthode (4 pts.). (à compléter)